In [1]:
# import dependencies
from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from base64 import b64decode, b64encode
import cv2
import numpy as np
import PIL
import io
import html
import time

In [2]:
# function to convert the JavaScript object into an OpenCV image
def js_to_image(js_reply):
  """
  Params:
          js_reply: JavaScript object containing image from webcam
  Returns:
          img: OpenCV BGR image
  """
  # decode base64 image
  image_bytes = b64decode(js_reply.split(',')[1])
  # convert bytes to numpy array
  jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
  # decode numpy array into OpenCV BGR image
  img = cv2.imdecode(jpg_as_np, flags=1)

  return img

# function to convert OpenCV Rectangle bounding box image into base64 byte string to be overlayed on video stream
def bbox_to_bytes(bbox_array):
  """
  Params:
          bbox_array: Numpy array (pixels) containing rectangle to overlay on video stream.
  Returns:
        bytes: Base64 image byte string
  """
  # convert array into PIL image
  bbox_PIL = PIL.Image.fromarray(bbox_array, 'RGBA')
  iobuf = io.BytesIO()
  # format bbox into png for return
  bbox_PIL.save(iobuf, format='png')
  # format return string
  bbox_bytes = 'data:image/png;base64,{}'.format((str(b64encode(iobuf.getvalue()), 'utf-8')))

  return bbox_bytes

In [3]:
# JavaScript to properly create our live video stream using our webcam as input
def video_stream():
  js = Javascript('''
    var video;
    var div = null;
    var stream;
    var captureCanvas;
    var imgElement;
    var labelElement;
    
    var pendingResolve = null;
    var shutdown = false;
    
    function removeDom() {
       stream.getVideoTracks()[0].stop();
       video.remove();
       div.remove();
       video = null;
       div = null;
       stream = null;
       imgElement = null;
       captureCanvas = null;
       labelElement = null;
    }
    
    function onAnimationFrame() {
      if (!shutdown) {
        window.requestAnimationFrame(onAnimationFrame);
      }
      if (pendingResolve) {
        var result = "";
        if (!shutdown) {
          captureCanvas.getContext('2d').drawImage(video, 0, 0, 640, 480);
          result = captureCanvas.toDataURL('image/jpeg', 0.8)
        }
        var lp = pendingResolve;
        pendingResolve = null;
        lp(result);
      }
    }
    
    async function createDom() {
      if (div !== null) {
        return stream;
      }

      div = document.createElement('div');
      div.style.border = '2px solid black';
      div.style.padding = '3px';
      div.style.width = '100%';
      div.style.maxWidth = '600px';
      document.body.appendChild(div);
      
      const modelOut = document.createElement('div');
      modelOut.innerHTML = "<span>Status:</span>";
      labelElement = document.createElement('span');
      labelElement.innerText = 'No data';
      labelElement.style.fontWeight = 'bold';
      modelOut.appendChild(labelElement);
      div.appendChild(modelOut);
           
      video = document.createElement('video');
      video.style.display = 'block';
      video.width = div.clientWidth - 6;
      video.setAttribute('playsinline', '');
      video.onclick = () => { shutdown = true; };
      stream = await navigator.mediaDevices.getUserMedia(
          {video: { facingMode: "environment"}});
      div.appendChild(video);

      imgElement = document.createElement('img');
      imgElement.style.position = 'absolute';
      imgElement.style.zIndex = 1;
      imgElement.onclick = () => { shutdown = true; };
      div.appendChild(imgElement);
      
      const instruction = document.createElement('div');
      instruction.innerHTML = 
          '<span style="color: red; font-weight: bold;">' +
          '0 = A, 1 = B, 2 = C, 3 = D, 4 = E';
      div.appendChild(instruction);
      instruction.onclick = () => { shutdown = true; };
      
      video.srcObject = stream;
      await video.play();

      captureCanvas = document.createElement('canvas');
      captureCanvas.width = 640; //video.videoWidth;
      captureCanvas.height = 480; //video.videoHeight;
      window.requestAnimationFrame(onAnimationFrame);
      
      return stream;
    }
    async function stream_frame(label, imgData) {
      if (shutdown) {
        removeDom();
        shutdown = false;
        return '';
      }

      var preCreate = Date.now();
      stream = await createDom();
      
      var preShow = Date.now();
      if (label != "") {
        labelElement.innerHTML = label;
      }
            
      if (imgData != "") {
        var videoRect = video.getClientRects()[0];
        imgElement.style.top = videoRect.top + "px";
        imgElement.style.left = videoRect.left + "px";
        imgElement.style.width = videoRect.width + "px";
        imgElement.style.height = videoRect.height + "px";
        imgElement.src = imgData;
      }
      
      var preCapture = Date.now();
      var result = await new Promise(function(resolve, reject) {
        pendingResolve = resolve;
      });
      shutdown = false;
      
      return {'create': preShow - preCreate, 
              'show': preCapture - preShow, 
              'capture': Date.now() - preCapture,
              'img': result};
    }
    ''')

  display(js)
  
def video_frame(label, bbox):
  data = eval_js('stream_frame("{}", "{}")'.format(label, bbox))
  return data

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os
import PIL
import pathlib
data_dir = pathlib.Path('/content/drive/Shareddrives/Team Koko/Dataset/partial')

stage_images_dict = {
    'A': list(data_dir.glob('A/*')),
    'B': list(data_dir.glob('B/*')),
    'C': list(data_dir.glob('C/*')),
    'D': list(data_dir.glob('D/*')),
    'E': list(data_dir.glob('E/*')),

}
stage_labels_dict = {
    'A': 0,
    'B': 1,
    'C': 2,
    'D': 3,
    'E': 4,

}

X, y = [], []

for stage_name, images in stage_images_dict.items():
    for image in images:
        img = cv2.imread(str(image))
        resized_img = cv2.resize(img,(128,128))
        X.append(resized_img)
        y.append(stage_labels_dict[stage_name])

X = np.array(X)
y = np.array(y)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

X_train_scaled = X_train / 255
X_test_scaled = X_test / 255

In [6]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras import models
from tensorflow.keras.layers import Dense,Dropout
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

base_model=MobileNet(weights='imagenet',
                     include_top=False, 
                     input_shape=(128, 128, 3))  # shape of RGB images 
data_augmentation = keras.Sequential(
  [
    layers.experimental.preprocessing.RandomRotation(0.1),
    layers.experimental.preprocessing.RandomZoom(0.1),
    # layers.experimental.preprocessing.RandomFlip(),
  ]
)

num_classes = 5

model = models.Sequential()
model.add(layers.BatchNormalization()),
model.add(data_augmentation) # Uncomment to add Data augmentation 
model.add(base_model)
model.add(layers.Flatten())
model.add(layers.Dense(2048, activation='relu'))  # Changed number of nodes from 1024 to 2048
model.add(Dropout(0.5))
model.add(layers.Dense(1024, activation='relu'))  # Changed number of nodes from 512 to 1024
model.add(Dropout(0.5))
model.add(layers.Dense(num_classes,activation='softmax'))

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])



# Included ", callbacks=[tensorboard]"
# 2.0: Included the use of "validation_data" in the model.fit
model.fit(X_train_scaled, y_train, epochs=40, validation_data=(X_test_scaled,y_test))

Epoch 1/40
15/15 [==============================] - 46s 3s/step - loss: 15.2956 - accuracy: 0.2310 - val_loss: 10.5160 - val_accuracy: 0.3733
Epoch 2/40
15/15 [==============================] - 39s 3s/step - loss: 4.6242 - accuracy: 0.5124 - val_loss: 2.9694 - val_accuracy: 0.5267
Epoch 3/40
15/15 [==============================] - 39s 3s/step - loss: 2.1130 - accuracy: 0.6507 - val_loss: 2.1351 - val_accuracy: 0.5933
Epoch 4/40
15/15 [==============================] - 39s 3s/step - loss: 1.3688 - accuracy: 0.7056 - val_loss: 1.8286 - val_accuracy: 0.6600
Epoch 5/40
15/15 [==============================] - 39s 3s/step - loss: 0.7522 - accuracy: 0.7939 - val_loss: 2.0492 - val_accuracy: 0.6400
Epoch 6/40
15/15 [==============================] - 39s 3s/step - loss: 0.5421 - accuracy: 0.8481 - val_loss: 1.9759 - val_accuracy: 0.6400
Epoch 7/40
15/15 [==============================] - 39s 3s/step - loss: 0.5391 - accuracy: 0.8288 - val_loss: 1.7268 - val_accuracy: 0.6933
Epoch 8/40
15/15 [

In [7]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization (BatchNo (None, 128, 128, 3)       12        
_________________________________________________________________
sequential (Sequential)      (None, 128, 128, 3)       0         
_________________________________________________________________
mobilenet_1.00_128 (Function (None, 4, 4, 1024)        3228864   
_________________________________________________________________
flatten (Flatten)            (None, 16384)             0         
_________________________________________________________________
dense (Dense)                (None, 2048)              33556480  
_________________________________________________________________
dropout (Dropout)            (None, 2048)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1024)             

In [8]:
import ipywidgets
prediction_widget = ipywidgets.Text(description='prediction')
score_widgets = []
for category in stage_labels_dict:
    score_widget = ipywidgets.FloatSlider(min=0.0, max=1.0, description=category, orientation='vertical')
    score_widgets.append(score_widget)

In [9]:
# start streaming video from webcam
video_stream()
# label for video
label_html = []
# initialze bounding box to empty
bbox = ''
count = 0 
while True:
    js_reply = video_frame(label_html, bbox)
    if not js_reply:
        break

    # convert JS response to OpenCV Image
    img = js_to_image(js_reply["img"])
    resized_img = cv2.resize(img,(128,128))
    resized_img_scaled = np.array(resized_img) / 255
    resized_img_scaled = resized_img_scaled.reshape(1 ,128, 128, 3)
    # print(resized_img.shape)
    label_html = "{}".format(np.argmax(model.predict(resized_img_scaled)))
    # print(model.predict(resized_img_scaled))


<IPython.core.display.Javascript object>

<h1 style="background-color:#76b900;"></h1>